In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from pathlib import Path
import flexiznam as flz
from cottage_analysis.analysis import spheres
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, get_session_list
from v1_depth_analysis.v1_manuscript_2023 import common_utils as plt_common_utils

In [ ]:
SAVE_ROOT = (
    Path(
        f"/Volumes/lab-znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver{VERSION}"
    )
    / "fig1"
)
SAVE_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
# Load data
# Load an example session
project = "hey2_3d-vision_foodres_20220101"
session_name = "PZAH8.2h_S20230116"
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=5,
    return_volumes=True,
)
suite2p_ds = flz.get_datasets_recursively(
    flexilims_session=flexilims_session,
    origin_name=session_name,
    dataset_type="suite2p_traces",
)
fs = list(suite2p_ds.values())[0][-1].extra_attributes["fs"]

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=None,
    conflicts="skip",
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)


In [ ]:
# concatenate neurons_df for all sessions
neurons_df_all = plt_common_utils.concatenate_all_neurons_df(
    flexilims_session=flexilims_session,
    session_list=get_session_list.get_all_sessions(
        project=project,
        mouse_list=[
            "PZAH6.4b",
            "PZAG3.4f",
            "PZAH8.2h",
            "PZAH8.2i",
            "PZAH8.2f",
            "PZAH10.2d",
            "PZAH10.2f",
        ],
        closedloop_only=False,
        openloop_only=False,
    ),
    filename="neurons_df.pickle",
    cols=[
        "roi",
        "is_depth_neuron",
        "depth_neuron_anova_p",
        "best_depth",
        "preferred_depth_closedloop",
        "depth_tuning_popt_closedloop",
        "depth_tuning_trials_closedloop_crossval",
        "preferred_depth_closedloop_crossval",
        "depth_tuning_test_rsq_closedloop",
        "depth_tuning_test_spearmanr_rval_closedloop",
        "depth_tuning_test_spearmanr_pval_closedloop",
    ],
    read_iscell=True,
    verbose=True,
)
neurons_df_all = neurons_df_all[neurons_df_all["iscell"] == 1]

# Load the depth tuning raster
results_all = pd.read_pickle(SAVE_ROOT / "results_all.pickle")
results_all = neurons_df_all[neurons_df_all["iscell"] == 1]


In [ ]:
# set font to arial
plt.rcParams["font.family"] = "Arial"
fontsize_dict = {"title": 7, "label": 7, "tick": 5, "legend": 5}
cm = 1 / 2.54
# Fig.1A: Motion parallax schema & equation, VR schema
# Fig.1B: Trial structure schema, Spheres with different sizes schema

# Fig.1C: Raster plot of an example neuron
EXAMPLE_ROI = 250
fig = plt.figure(figsize=(18 * cm, 18 * cm))
depth_selectivity.plot_raster_all_depths(
    trials_df=trials_df_example,
    roi=EXAMPLE_ROI,
    is_closed_loop=True,
    corridor_length=6,
    blank_length=3,
    nbins=60,
    vmax=3,
    plot=True,
    cbar_width=0.01,
    fontsize_dict=fontsize_dict,
    position=[0, 0.75, 0.4, 0.15],
)

# Fig.1D: Example PSTH of a single neuron
fig.add_axes([0.5, 0.75, 0.2, 0.15])
depth_selectivity.plot_PSTH(
    trials_df=trials_df_example,
    roi=EXAMPLE_ROI,
    is_closed_loop=True,
    corridor_length=6,
    blank_length=3,
    nbins=60,
    frame_rate=fs,
    fontsize_dict=fontsize_dict,
    linewidth=1,
    legend_on=True,
    show_ci=False,
)

# Fig.1E: Example depth tuning curve & PSTH of a single neuron
fig.add_axes([0.8, 0.75, 0.15, 0.15])
depth_selectivity.plot_depth_tuning_curve(
    neurons_df=neurons_df_example,
    trials_df=trials_df_example,
    roi=EXAMPLE_ROI,
    rs_thr=None,
    plot_fit=False,
    linewidth=2,
    linecolor="k",
    fit_linecolor="r",
    closed_loop=1,
    fontsize_dict=fontsize_dict,
)

# Fig.1F: Example depth tuning curve & PSTH of 3 other neurons
EXAMPLE_ROIS = [174, 249, 54, 638, 86, 88]
plot_cols = 3
for i, roi in enumerate(EXAMPLE_ROIS):
    fig.add_axes([i * 0.166, 0.54, 0.13, 0.13])
    depth_selectivity.plot_depth_tuning_curve(
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=None,
        plot_fit=False,
        linewidth=2,
        linecolor="k",
        fit_linecolor="r",
        closed_loop=1,
        fontsize_dict=fontsize_dict,
    )
    if i != 0:
        plt.ylabel("")

# Fig.1G: Histogram of proportion of depth-tuned neurons;
# preferred depths of all depth-tuned neurons;
# Raster plot of preferred depths of all neurons (sorted by preferred depth)
fig.add_axes([0, 0.3, 0.2, 0.15])
depth_selectivity.plot_depth_neuron_perc_hist(
    results_df=neurons_df_all,
    numerator_filter=(
        (neurons_df_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    ),
    denominator_filter=None,
    bins=np.arange(0, 1.0, 0.05),
    fontsize_dict=fontsize_dict,
)

fig.add_axes([0.28, 0.3, 0.2, 0.15])
depth_selectivity.plot_preferred_depth_hist(
    results_df=neurons_df_all[
        neurons_df_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05
    ],
    use_col="preferred_depth_closedloop_crossval",
    nbins=20,
    fontsize_dict=fontsize_dict,
)
fig.savefig(SAVE_ROOT / "fig1.pdf")

# depth_selectivity.plot_psth_raster(
#     fig=fig,
#     results_df=results_all,
#     depth_list=np.geomspace(5, 640, num=8).astype(int),
#     use_cols=["preferred_depth_crossval", "psth_crossval", "preferred_depth_rsq"],
#     depth_rsq_thr=0.04,
#     plot_x=0.8,
#     plot_y=0.2,
#     plot_width=0.25,
#     plot_height=0.3,
#     fontsize_dict={"title": 15, "label": 10, "tick": 10},
# )

# Visually responsive neurons: 0.5s before the onset vs average of the whole trial (look at mean response)
# fig.savefig(SAVE_ROOT / "fig1.pdf", bbox_inches="tight", transparent=True)
# fig.savefig(SAVE_ROOT / "fig1.png", bbox_inches="tight", transparent=True, dpi=300)


In [ ]:
# CHECK FALSE POSITIVES ARE 5 % (SHUFFLED DISTRIBUTION FOR A FEW SESSIONS)
# put 5 depths raster in the supp
# CAD plot of stimuli (softwares that can draw 3d stuff, photoshop)
# MINOR: histogram: add dash line indicating the depth range in the experiment

## SUPP


In [ ]:
from matplotlib_venn import venn3

depth_neurons = neurons_df_6f[
    (neurons_df_6f["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (neurons_df_6f["iscell"] == 1)
].index.values
visually_responsive = neurons_df_6f[
    (neurons_df_6f["visually_responsive"] == 1) & (neurons_df_6f["iscell"] == 1)
].index.values
all_neurons = neurons_df_6f[neurons_df_6f["iscell"] == 1].index.values
venn3(
    [set(depth_neurons), set(visually_responsive), set(all_neurons)],
    ("Depth-tuned", "Visually responsive", "All neurons"),
)
print(
    len(visually_responsive) / len(all_neurons), len(depth_neurons) / len(all_neurons)
)

In [ ]:
fig = plt.figure()
# Fig.1F: Example depth tuning curve & PSTH of 3 other neurons
EXAMPLE_ROIS = [249, 54, 88]
for i, roi in enumerate(EXAMPLE_ROIS):
    if i == len(EXAMPLE_ROIS) - 1:
        legend_on = True
    else:
        legend_on = False
    depth_selectivity.plot_PSTH(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        plot_x=0.05 + 0.25 * (i + 1),
        plot_y=0.67,
        plot_width=0.2,
        plot_height=0.18,
        fontsize_dict={"title": 15, "label": 10, "tick": 10, "legend": 5},
        linewidth=2,
        legend_on=legend_on,
    )
    if i != 0:
        plt.ylabel("")

    ax2 = depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=2,
        linecolor="r",
        fit_linecolor="r",
        closed_loop=1,
        plot_x=0.05 + 0.25 * (i + 1) + 0.12,
        plot_y=0.79,
        plot_width=0.08,
        plot_height=0.08,
        fontsize_dict={"title": 15, "label": 10, "tick": 10},
    )
    plt.ylabel("")
    plt.xlabel("")
    plt.xticks([])

    for rs_thr, rs_thr_max, still_only, still_time, label, linecolor in zip(
        [None, 0.05, None],
        [None, 0.2, 0.05],
        [False, False, True],
        [0, 0, 1],
        ["All", "Walking\n(5-20cm/s)", "Not running\nfor 1s before"],
        ["b", "orange", "gray"],
    ):
        depth_selectivity.plot_depth_tuning_curve(
            fig=fig,
            neurons_df=neurons_df_example,
            trials_df=trials_df_example,
            roi=roi,
            rs_thr=rs_thr,
            rs_thr_max=rs_thr_max,
            still_only=still_only,
            still_time=still_time,
            plot_fit=False,
            linewidth=2,
            linecolor=linecolor,
            fit_linecolor=None,
            closed_loop=1,
            overwrite_ax=False,
            ax=ax2,
            plot_x=0.05 + 0.25 * (i + 1) + 0.12,
            plot_y=0.79,
            plot_width=0.08,
            plot_height=0.08,
            label=label,
            fontsize_dict={"title": 15, "label": 10, "tick": 10},
        )
        plt.ylabel("")
        plt.xlabel("")
        plt.xticks([])
    if i == len(EXAMPLE_ROIS) - 1:
        plt.legend(
            frameon=False, handlelength=1, bbox_to_anchor=[0.51, 0.51], fontsize=5
        )

In [ ]:
# Depth tuning curve & PSTH of different running states
fig = plt.figure()
rois = [250, 249, 54, 88]
for i, roi in enumerate(rois):
    # depth tuning curve
    ax = depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=0.2,
        plot_fit=False,
        linewidth=2,
        linecolor="r",
        fit_linecolor="r",
        closed_loop=1,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=1,
        plot_width=0.2,
        plot_height=0.2,
        label="Running (> 20cm/s)",
        fontsize_dict={"title": 15, "label": 10, "tick": 10},
    )
    if i != 0:
        plt.ylabel("")

    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=None,
        rs_thr_max=None,
        still_only=False,
        plot_fit=False,
        linewidth=2,
        linecolor="b",
        fit_linecolor="r",
        closed_loop=1,
        overwrite_ax=False,
        ax=ax,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=1,
        plot_width=0.2,
        plot_height=0.2,
        label="All",
        fontsize_dict={"title": 15, "label": 10, "tick": 10},
    )
    if i != 0:
        plt.ylabel("")

    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=0.05,
        rs_thr_max=0.2,
        still_only=False,
        plot_fit=False,
        linewidth=2,
        linecolor="orange",
        fit_linecolor="r",
        closed_loop=1,
        overwrite_ax=False,
        ax=ax,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=1,
        plot_width=0.2,
        plot_height=0.2,
        label="Walking (5-20cm/s)",
        fontsize_dict={"title": 15, "label": 10, "tick": 10},
    )
    if i != 0:
        plt.ylabel("")

    depth_selectivity.plot_depth_tuning_curve(
        fig=fig,
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=None,
        rs_thr_max=0.05,
        still_only=True,
        still_time=1,
        frame_rate=15,
        plot_fit=False,
        linewidth=2,
        linecolor="gray",
        fit_linecolor="r",
        closed_loop=1,
        overwrite_ax=False,
        ax=ax,
        plot_x=0 + 0.15 * (i + 1),
        plot_y=1,
        plot_width=0.2,
        plot_height=0.2,
        label="Not running for 1s before",
        fontsize_dict={"title": 15, "label": 10, "tick": 10},
    )
    if i != 0:
        plt.ylabel("")
    if i == len(rois) - 1:
        plt.legend(fontsize=5)

    # PSTH
    depth_selectivity.plot_PSTH(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        rs_thr_min=0.2,
        still_only=False,
        still_time=0,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=0.47,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict={"title": 15, "label": 10, "tick": 10, "legend": 5},
        linewidth=2,
        legend_on=False,
    )
    if i != 0:
        plt.ylabel("")
    if i == 0:
        plt.title("Running (> 20cm/s)")
    ylim = plt.gca().get_ylim()
    plt.xlabel("")

    depth_selectivity.plot_PSTH(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        rs_thr_min=None,
        still_only=False,
        still_time=0,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=0.7,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict={"title": 15, "label": 10, "tick": 10, "legend": 5},
        linewidth=2,
        legend_on=False,
    )
    if i != 0:
        plt.ylabel("")
    if i == 0:
        plt.title("All frames")
    plt.ylim(ylim)
    plt.xlabel("")

    depth_selectivity.plot_PSTH(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        rs_thr_min=0.05,
        rs_thr_max=0.2,
        still_only=False,
        still_time=0,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=0.24,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict={"title": 15, "label": 10, "tick": 10, "legend": 5},
        linewidth=2,
        legend_on=False,
    )
    if i != 0:
        plt.ylabel("")
    if i == 0:
        plt.title("Walking (5-20cm/s)")
    plt.ylim(ylim)
    plt.xlabel("")

    depth_selectivity.plot_PSTH(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        is_closed_loop=True,
        max_distance=6,
        nbins=20,
        frame_rate=15,
        rs_thr_min=None,
        rs_thr_max=0.05,
        still_only=True,
        still_time=1,
        plot_x=0 + 0.25 * (i + 1),
        plot_y=0,
        plot_width=0.15,
        plot_height=0.1,
        fontsize_dict={"title": 15, "label": 10, "tick": 10, "legend": 5},
        linewidth=2,
        legend_on=False,
    )
    if i != 0:
        plt.ylabel("")
    if i == 0:
        plt.title("Not running for 1s before")
    plt.ylim(ylim)

# QUANTIFY: select depth-tuned neurons based on all frames, compare max response from running / non-running